# MCS Algorithms for Order and Chaos Game

### Useful Links
- [Order and Chaos - Wikipedia](https://en.wikipedia.org/wiki/Order_and_Chaos)
- [Order and Chaos - Game simulator](https://ludii.games/details.php?keyword=Order%20and%20Chaos)

**Abstract :** For this project we interested about Order and Chaos game who is very challenging because of is rules. Like in tic-tac-toe players allowed between to state O or X. His particularity come from the fact that both players allowed to play O and X, so the main objective for player 1 and player 2 are different. The first want to make a winning line of 5 successivly same symbole and the other want to make a draw. For this game we tried differente Monte carlo tree search algorithms :
- UCB ?? for ..
- RAVE ?? because ..
- ETC.. TO DO

## Import

In [11]:
import numpy as np
import numpy.random as random
from copy import deepcopy

## Constant inputs

In [3]:
Dx = 6
Dy = 6
Empty = '-'

## Define the Game

### Move class

In [4]:
class Move(object):
    def __init__(self, x, y, symbol):
        self.x = x
        self.y = y
        self.symbol = symbol
        
    def valid (self, board):
        # Move in the board
        if self.x >= Dx or self.y >= Dy or self.x < 0 or self.y <0:
            return False
        # Move in a free space
        if board.board[self.x][self.y] != Empty:
            return False
        
        return True

### Class Board

In [12]:
class Board(object):
    def __init__(self):
        #self.history = {'Order' : [], 'Chaos' : []}
        self.turn = 'Order' #Order always start the game
        self.board = [[Empty for _ in range(Dx)] for _ in range(Dy)]
        self.board = np.array(self.board)
        self.nb_moves = 0
        
    def legalMoves(self):
        moves = []
        for i in range (0, Dx):
            for j in range (0, Dy):
                for symb in ['X', 'O']:
                    m = Move (i, j, symb)
                    if m.valid (self):
                        moves.append (m)
                    
        return moves
    
    def verif_array(self, array):
        if (np.char.count(array, '-').sum() <= 1) and (len(np.unique(array[:5]))==1 or len(np.unique(array[1:]))==1):
            return True
        else:
            return False
        
    def win(self):
        # Horizontal win
        for row in self.board:
            if self.verif_array(row) == True:
                return True
            
        # Vertical win
        for col in self.board.T:
            if self.verif_array(col) == True:
                return True
            
        # Diagonal win
        for i in [-1, 0, 1]:
            diag = np.diagonal(self.board, offset=i)
            if (len(diag)==6) and (self.verif_array(diag) == True) or (np.char.count(diag, '-').sum() < 1) and (len(np.unique(diag))==1):
                return True
            
            diag_opposite = np.flipud(self.board).diagonal(offset=i)
            if (len(diag_opposite)==6) and (self.verif_array(diag_opposite) == True) or (np.char.count(diag_opposite, '-').sum() < 1) and (len(np.unique(diag_opposite))==1):
                return True
            
        return False
        
        
    def draw(self):
        if self.nb_moves < Dx*Dy:
            return False
        if self.nb_moves >= Dx*Dy and not self.win():
            return True
        return False
    
    def score(self):
        if self.draw():
            return 0
        if self.win():
            return 1
    
    def terminal(self):
        if not self.win() and not self.draw():
            return False
        return True
    
    def __repr__(self):
        return "\n".join(" ".join(row) for row in self.board)
        
    def play (self, move:Move):
        change_player = {'Order' : 'Chaos', 'Chaos' : 'Order'}
        if move.valid(self):
            self.board[move.x][move.y] = move.symbol
            self.turn = change_player[self.turn]
            self.nb_moves += 1

    def playout (self):
        while (True):
            moves = self.legalMoves()
            if self.terminal():
                return self.score()
            n = random.randint(0, len (moves) - 1)
            self.play(moves [n])

## Test the game

### Manually

In [15]:
game = Board()
game.play(Move(0, 0, 'X'))
game.play(Move(1, 1, 'X'))
game.play(Move(2, 2, 'O'))
game.play(Move(3, 3, 'X'))
game.play(Move(4, 4, 'X'))
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())
game.play(Move(2, 0, 'O'))
game.play(Move(2, 1, 'O'))
game.play(Move(2, 3, 'O'))
game.play(Move(2, 4, 'O'))
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())

X - - - - -
- X - - - -
- - O - - -
- - - X - -
- - - - X -
- - - - - -
Win: False
Draw: False
End: False
X - - - - -
- X - - - -
O O O O O -
- - - X - -
- - - - X -
- - - - - -
Win: True
Draw: False
End: True


### Playout

In [23]:
game = Board()
game.playout()
print(game)
print("Win:", game.win())
print("Draw:", game.draw())
print("End:", game.terminal())

O O O O X X
X O X O - -
X O O O X O
O O X X O O
X X X X O X
- X X X X X
Win: True
Draw: False
End: True


### Win rate after 1000 playout

In [27]:
wins = []
for _ in range(1000):
    game = Board()
    score = game.playout()
    wins.append(score)
    
print(f"Win rate after 1000 playout: {sum(wins)/1000:.4f}")

Win rate after 1000 playout: 0.8100


# Define algorithms

## UCB